In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arjuntejaswi/plant-village")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/plant-village


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

datapath = '/kaggle/input/plant-village'
dataset = datasets.ImageFolder(root = datapath, transform = transform)

In [4]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
print(len(train_dataset), len(test_dataset))

16510 4128


In [5]:
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle = True)
test_data_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

In [6]:
class CustomCNN(nn.Module):
    def __init__(self, out_features):
        super(CustomCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # (3, 64, 112, 112)

            nn.Conv2d(64, 128, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # (64, 128, 56, 56)

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride = 2), # (128, 256, 28, 28)

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # (256, 512, 14, 14)

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # (512, 512, 7, 7)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(2048, out_features)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


cnn_model = CustomCNN(out_features=15)
cnn_model.to(device)
print(cnn_model)


CustomCNN(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
   

In [7]:
def training(model, train_data_loader, test_data_loader, epochs):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr = 0.01)
    criterion = nn.CrossEntropyLoss()

    for e in range(epochs):
        running_loss, total, correct = 0.0, 0, 0
        for features, lables in train_data_loader:
            features, lables = features.to(device), lables.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, lables)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += lables.size(0)
            correct += predicted.eq(lables).sum().item()

        train_acc = 100*correct/total
        train_loss = running_loss/len(train_data_loader)

        test_acc, test_loss, preds, lables = evaluate(model, test_data_loader, criterion)
        precision = precision_score(lables, preds, average='macro')
        recall = recall_score(lables, preds, average='macro')

        print(f"epochs: {e+1}/{epochs}, training loss: {train_loss: .4f}, training accuracy: {train_acc: .4f}, testing loss: {test_loss: .4f}, testing accuracy: {test_acc: .4f}, precision: {precision}, recall: {recall}")

def evaluate(model, test_data_loader, criterion):
    model.eval()
    running_loss, total, correct = 0.0, 0, 0

    all_preds = []
    all_lables = []

    with torch.no_grad():
        for features, lables in test_data_loader:
            features, lables = features.to(device), lables.to(device)
            output = model(features)
            loss = criterion(output, lables)

            running_loss += loss.item()
            _, predicted = output.max(1)
            total += lables.size(0)
            correct += predicted.eq(lables).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_lables.extend(lables.cpu().numpy())

    test_acc = 100*correct/total
    test_loss = running_loss/len(test_data_loader)

    return test_acc, test_loss, all_preds, all_lables

training(cnn_model, train_data_loader, test_data_loader, 5)

epochs: 1/5, training loss:  0.0053, training accuracy:  99.8062, testing loss:  0.0000, testing accuracy:  100.0000, precision: 1.0, recall: 1.0
epochs: 2/5, training loss:  0.0000, training accuracy:  100.0000, testing loss:  0.0000, testing accuracy:  100.0000, precision: 1.0, recall: 1.0
epochs: 3/5, training loss:  0.0000, training accuracy:  100.0000, testing loss:  0.0000, testing accuracy:  100.0000, precision: 1.0, recall: 1.0
epochs: 4/5, training loss:  0.0000, training accuracy:  100.0000, testing loss:  0.0000, testing accuracy:  100.0000, precision: 1.0, recall: 1.0
epochs: 5/5, training loss:  0.0000, training accuracy:  100.0000, testing loss:  0.0000, testing accuracy:  100.0000, precision: 1.0, recall: 1.0


In [10]:
torch.save(cnn_model.state_dict(), 'cnn_model.pth')